In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import sys

dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')

from interpolazione import Interpolazione

dir_path = os.path.abspath('')
FILE = dir_path + '/data/' + 'RLC sovrasmorzato.csv'
fr = pd.read_csv(FILE)

In [ ]:
t = np.array(fr['t (us)']) * 10**-6
V = np.array(fr['V (Volt)']) * 10**-3